In [1]:
%pip install py7zr


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import requests
import py7zr
import os
import io
import tempfile

def getUrl(annee):
    return f"https://data.drees.solidarites-sante.gouv.fr/api/v2/catalog/datasets/708_bases-statistiques-sae/attachments/sae_{annee}_bases_statistiques_formats_sas_csv_7z"

def csvName(file, annee):
    return f"SAE {annee} Bases statistiques - formats SAS-CSV/Bases statistiques/Bases CSV/{file}_{annee}r.csv"

output = "Data/SAE/"
url = getUrl(2019)
csvPERINAT = csvName("PERINAT", 2019)
csvID = csvName("ID", 2019)

def openFile(csv, encodage):
    response = requests.get(url)
    response.raise_for_status()

    with io.BytesIO(response.content) as f:
        with py7zr.SevenZipFile(f, mode='r') as z:
            if csv in z.getnames():
                with tempfile.TemporaryDirectory() as temp_dir:
                    z.extract(targets=[csv], path=temp_dir)
                    extracted_path = os.path.join(temp_dir, csv)
                    df = pd.read_csv(extracted_path,sep=";",encoding=encodage)

                    return df
            else:
                print(f"{csv} not found in the archive.")
                return None

dfFull = openFile(csvPERINAT, "utf-8")
dfPERINAT = dfFull[["FI", "PRIS", "IVG", "IVGN_1", "IVGME", "IVG1214", "CONV", "IMG"]]
dfPERINAT = dfPERINAT[dfPERINAT["PRIS"] == 1.0]

dfID = openFile(csvID, "latin1")
dfID["FI"] = dfID["fi"]

dfMerged = pd.merge(
    dfPERINAT,
    dfID[["FI", "rs", "stj", "stjr", "cat", "catr", "dep"]],
    on="FI",
    how="left" 
)
dfMerged["Annee"] = "2019"
        


In [19]:
dfMerged

,FI,PRIS,IVG,IVGN_1,IVGME,IVG1214,CONV,IMG,rs,stj,stjr,cat,catr,dep,Annee
0,010000024,1.0,550.0,525.0,414.0,38.0,NaN,29.0,CENTRE HOSPITALIER DE FLEYRIAT,13,1,355,PUB2,01,2019
1,010000032,1.0,88.0,91.0,66.0,2.0,0.0,1.0,CENTRE HOSPITALIER DE BELLEY,13,1,355,PUB2,01,2019
2,010005239,1.0,91.0,63.0,78.0,0.0,NaN,0.0,CH DU HAUT BUGEY - GEOVREISSET,14,1,355,PUB2,01,2019
3,020000162,1.0,399.0,399.0,306.0,18.0,1.0,9.0,CH SAINT-QUENTIN,13,1,355,PUB2,02,2019
4,020000394,1.0,244.0,235.0,203.0,11.0,NaN,5.0,CH LAON,13,1,355,PUB2,02,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,970400073,1.0,390.0,100.0,297.0,33.0,NaN,17.0,GHER (SAINT-BENOIT),14,1,355,PUB2,9D,2019
547,970404844,1.0,93.0,132.0,0.0,2.0,NaN,0.0,CLINIQUE SAINT VINCENT,95,2,365,PLU4,9D,2019
548,970462024,1.0,210.0,170.0,158.0,7.0,NaN,5.0,CLINIQUE JEANNE D'ARC,95,2,365,PLU4,9D,2019
549,970462107,1.0,263.0,374.0,261.0,3.0,0.0,8.0,CLINIQUE STE-CLOTILDE,95,2,365,PLU4,9D,2019
